In [1]:
from __future__ import print_function
import torch

In [10]:
x = torch.empty(5,3)
print(x)

tensor([[4.4684e+01, 4.5569e-41, 1.2702e-36],
        [3.0847e-41, 2.9858e-12, 4.5569e-41],
        [0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00]])


In [11]:
x[0][0]

tensor(44.6843)

In [15]:
x = torch.zeros(5,3, dtype=torch.long)
print (x)

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])


In [9]:
x[0][0]

tensor(0)

In [12]:
x.size()

torch.Size([5, 3])

In [13]:
type(x.size())

torch.Size

In [16]:
y = torch.rand(5,3)
print (x+y)

tensor([[0.2041, 0.7427, 0.4943],
        [0.7445, 0.0842, 0.5200],
        [0.9484, 0.2261, 0.3646],
        [0.9181, 0.5371, 0.8445],
        [0.4073, 0.2463, 0.8669]])


In [20]:
y.view(15)

tensor([0.2041, 0.7427, 0.4943, 0.7445, 0.0842, 0.5200, 0.9484, 0.2261, 0.3646,
        0.9181, 0.5371, 0.8445, 0.4073, 0.2463, 0.8669])

In [21]:
y[1][2].item()

0.5199916958808899

In [27]:
x = torch.ones(2,2, requires_grad=True)
y = torch.rand(2,2, requires_grad=True)

In [28]:
x, y

(tensor([[1., 1.],
         [1., 1.]], requires_grad=True),
 tensor([[0.5923, 0.1406],
         [0.4570, 0.6509]], requires_grad=True))

In [30]:
x.requires_grad = False

In [31]:
out = x + y

In [32]:
out

tensor([[1.5923, 1.1406],
        [1.4570, 1.6509]], grad_fn=<AddBackward0>)

In [33]:
out.grad_fn

In [37]:
out = out * out * 3
out = out.mean()

print (out)

tensor(135.3769, grad_fn=<MeanBackward0>)


In [38]:
x.requires_grad

False

In [39]:
out.backward()

/home/vansh/programs/anaconda3/lib/python3.8/site-packages/torch/autograd/__init__.py:130: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729062494/work/c10/cuda/CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(


In [42]:
print (x.grad, y.grad)

None tensor([[108.9954,  40.0615],
        [ 83.5150, 121.4954]])


In [43]:
x.requires_grad_ = True

In [44]:
out.backward()
print (x.grad, y.grad)

RuntimeError: Trying to backward through the graph a second time, but the saved intermediate results have already been freed. Specify retain_graph=True when calling backward the first time.

In [58]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [59]:
params = list(net.parameters())
print (len(params), params[0].size())

10 torch.Size([6, 1, 3, 3])


In [60]:
input = torch.randn(2,1,32,32)
out = net(input)
print (out)

tensor([[ 0.0950,  0.0950, -0.1015,  0.0787,  0.0493, -0.1208, -0.0033,  0.0450,
          0.0210, -0.0083],
        [ 0.0915,  0.0879, -0.0742,  0.1018,  0.0339, -0.1266, -0.0391,  0.0541,
          0.0092, -0.0274]], grad_fn=<AddmmBackward>)


In [49]:
net.zero_grad()
out.backward(torch.randn(2,10))

In [61]:
target = torch.randn(2,10)

criterion = nn.MSELoss()

loss = criterion(out, target)
print (loss)

tensor(0.6725, grad_fn=<MseLossBackward>)


In [53]:
dir(loss.grad_fn)

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_register_hook_dict',
 'metadata',
 'name',
 'next_functions',
 'register_hook',
 'requires_grad']

In [56]:
loss.grad_fn.next_functions[0][0].next_functions[0][0]

In [62]:
net.zero_grad()
print (net.conv2.bias.grad, net.conv2.bias)

loss.backward()

print (net.conv2.bias.grad, net.conv2.bias)

None Parameter containing:
tensor([ 0.0516,  0.0137,  0.0030, -0.1157,  0.0861, -0.0003,  0.0225,  0.0471,
        -0.0121,  0.0134, -0.0613, -0.1221, -0.0833, -0.0494, -0.1158,  0.0300],
       requires_grad=True)
tensor([ 0.0063,  0.0080,  0.0151,  0.0010, -0.0065, -0.0128,  0.0000, -0.0045,
        -0.0082,  0.0073, -0.0034, -0.0041, -0.0034, -0.0017, -0.0015,  0.0056]) Parameter containing:
tensor([ 0.0516,  0.0137,  0.0030, -0.1157,  0.0861, -0.0003,  0.0225,  0.0471,
        -0.0121,  0.0134, -0.0613, -0.1221, -0.0833, -0.0494, -0.1158,  0.0300],
       requires_grad=True)


In [63]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)

optimizer.zero_grad()
output = net(input)
loss = criterion(output ,target)
optimizer.step()

In [65]:
net.conv1.requires_grad_ = False

input = torch.randn(2,1,32,32)
target = torch.randn(2,10)

output = net(input)
loss = criterion(output, target)
optimizer.step()

In [66]:
a = torch.empty(1,2)

In [93]:
x = torch.ones(10, requires_grad=True)
y = torch.ones(10, requires_grad=True)

c = torch.stack([x,y])

In [83]:
c

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], grad_fn=<StackBackward>)

In [88]:
d = c * c * c - torch.rand(10)
out = d.mean()

In [89]:
print(x.grad)

None


In [91]:
out.backward()

In [92]:
print (x.grad)

tensor([0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500,
        0.1500])


In [95]:
dots = torch.empty(10)

for i in range(10):
    dots[i] = x.dot(y)


newout = dots - torch.rand(10)
newoutmean = newout.mean()


In [97]:
print(x.grad)

None


In [98]:
newoutmean.backward()

In [99]:
print(x.grad)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])


In [1]:
print ('hi')

hi


In [3]:
assert(True)

In [4]:
assert(False)

AssertionError: 

In [8]:
assert False, "must have nonzero hidden layers"

AssertionError: must have nonzero hidden layers

In [6]:
print(assert.__doc__)

SyntaxError: invalid syntax (<ipython-input-6-d71de5fd341d>, line 1)

In [10]:
import numpy as np


n = np.array(list(range(100,150)))

n[[4,5,6,8,4,4,]]

array([104, 105, 106, 108, 104, 104])

In [11]:
a = torch.zeros(10,2)

In [12]:
a[0]

tensor([0., 0.])

In [13]:
a[0].long()

tensor([0, 0])

In [2]:
u = torch.rand(2,3, requires_grad=True)


In [3]:
u.requires_grad_(True)

tensor([[0.7842, 0.4726, 0.1368],
        [0.7547, 0.9469, 0.1382]], requires_grad=True)

In [4]:
u

tensor([[0.7842, 0.4726, 0.1368],
        [0.7547, 0.9469, 0.1382]], requires_grad=True)

In [5]:
u.requires_grad_(False)

tensor([[0.7842, 0.4726, 0.1368],
        [0.7547, 0.9469, 0.1382]])

In [6]:
u

tensor([[0.7842, 0.4726, 0.1368],
        [0.7547, 0.9469, 0.1382]])